# Modelling a pincell

The purpose of this document is to describe the material and geometry modeling contained in :mod:`hydep`. Users should refer to [the geometry scope](../scope.rst#geometry-modeling) and [the API reference](../api/geometry.rst) to understand the motivation behind the library design.

We will start with building a single 2D pin cell, with fuel, gap, clad, and moderator region.

In [1]:
import math

import hydep

## Materials

There are two material classes provided in ``hydep``: ``Material`` and ``BurnableMaterial``. The latter behaves identically to the former, with the primary distinction that its compositions will be evolved through the simulation. When creating a material, one must supply either the mass density of the material in grams/cm$^3$, or atom density, in atoms/b-cm. Optionally, one can also define temperature (in K) and volumes (in cm$^3$), either at construction or later as attributes. **NOTE** Burnable materials must have a volume prior to running the simulation.

In [2]:
water = hydep.Material("water", mdens=0.75)
water.temperature = 600

Populating the material with isotopes is done by treating the material like a dictionary. Keys can be either string isotope names or integers representing the ``ZAI`` identifier for a given isotope. ``Z`` denotes number of protons, ``A`` protons and neutrons, and ``I`` a metastable flag with 0 indicating a ground state isotope and postive number the metastable state. The corresponding values are the individual atom densities in atoms/b-cm for each isotope. This differs from atomic fractions, as will be demonstrated with water

In [3]:
water["H1"] = 5.01543e-2
water[80160] = 2.50771e-2

We can indicate that thermal scattering $S(\alpha,\beta)$ libraries for hydrogen in water should be used for our moderator 

In [4]:
water.addSAlphaBeta("HinH20")

In [5]:
water

Material 1 water 7.50000e-01 [g/cc] at 600.00000 K
  H1: 5.01543e-02
  O16: 2.50771e-02
S(a,b): ['HinH20']

In [6]:
gap = hydep.Material("helium", mdens=1.5981e-3, temperature=600)

In [7]:
gap["He4"] = 2.4004e-4

In [8]:
clad = hydep.Material("zirc4", mdens=6.6, temperature=600)

In [9]:
clad[80160] = 3.07430e-04
clad[80170] = 1.17110e-07
clad[240500] = 3.29620e-06
clad[240520] = 6.35640e-05
clad[240530] = 7.20760e-06
clad[240540] = 1.79410e-06
clad[260540] = 8.66990e-06
clad[260560] = 1.3610e-04
clad[260570] = 3.14310e-06
clad[260580] = 4.18290e-07
clad[400900] = 2.18270e-02
clad[400910] = 4.760e-03
clad[400920] = 7.27580e-03
clad[400940] = 7.37340e-03
clad[400960] = 1.18790e-03
clad[501120] = 4.67350e-06
clad[501140] = 3.17990e-06
clad[501150] = 1.63810e-06
clad[501160] = 7.00550e-05
clad[501170] = 3.70030e-05
clad[501180] = 1.16690e-04
clad[501190] = 4.13870e-05
clad[501200] = 1.56970e-04
clad[501220] = 2.23080e-05
clad[501240] = 2.78970e-05

Here we create a burnable fuel corresponding to 3.5 wt. % enriched UO2 fuel at a temperature of 900 K

In [10]:
fuel = hydep.BurnableMaterial("fuel", mdens=10.4, temperature=900)

In [11]:
fuel[80160] = 4.63917160e-02
fuel[922340] = 9.3422610e-06
fuel[922350] = 1.0452130e-03
fuel[922360] = 4.78757760e-06
fuel[922380] = 2.2145310e-02

<div class="alert alert-info">
    
**Note**

`hydep` does not provide any helper utilities for adding elements by enrichment or with other units
    
</div>

## Geometry

As mentioned in the package limitations, ``hydep`` is not a general purpose neutronics modeling utility. Instead, a restricted set of geometries is supported, primarily focused on LWRs (e.g. annular fuel pins, Cartesian lattices, etc.) For more information and reasoning, please see [the geometry scope](../scope.rst#geometry-modeling).

We will define the basic geometry using the following radii, and set the ``volume`` attribute of the fuel to represent the cross-sectional area

In [12]:
rFuel = 0.39218
rGap = 0.40005
rClad = 0.45720

In [13]:
fuel.volume = math.pi * rFuel**2

The fuel pin is defined with a series of outer radii, materials that fill the space between each radial ring, and an outer material. This outer material extends from the largest radius to infinity, unless explicitely bounded.

In [14]:
pin = hydep.Pin([rFuel, rGap, rClad], [fuel, gap, clad], outer=water)

In [15]:
pin

Pin 1 
(0.39218, 0.40005, 0.4572)
4 fuel, 2 helium, 3 zirc4
1 water

Now we place this pin as the "root universe" in our `Model`, which serves as a more compact interface between the geometry and the rest of the package

In [16]:
model = hydep.Model(pin)

Currently our problem is completely unbounded, since both our `Model` and its root universe do not have boundaries

In [17]:
pin.bounds

In [18]:
model.bounds

Some of the geometric structures can infer their boundaries, but this is not always the case. Without further information, the `Pin` does not know how far the outer region should extend. We can indicate that our model should span a pin pitch of 1.2 cm, and be unbounded axially by setting `Model.bounds = (x, y, z) = ((x_lower, x_upper), (y_lower, y_upper), (z_lower, z_upper))`

In [19]:
model.bounds = (
    (-0.6, 0.6),
    (-0.6, 0.6),
    None,
)

The special value of `bounds.z = None` denotes infinite axial boundaries

In [20]:
model.bounds

Boundaries((-0.6, 0.6), (-0.6, 0.6), (-inf, inf))

In [21]:
model.bounds.z

(-inf, inf)

<div class="alert alert-info">

**Note**

`hydep` does not contain any visualization routines nor model verification checks

</div>

## Depletion

Two classes are primarily responsible for defining the depletion schedule, and the paths by which isotopes are transmuted. 

First a `DepletionChain`, heavily inspired by 
[the OpenMC depletion API](https://docs.openmc.org/en/latest/pythonapi/generated/openmc.deplete.Chain.html#openmc.deplete.Chain), 
is used to describe the decay and reaction channels for a collection of isotopes. This `DepletionChain` can be built using the same XML input files, up to those supported by at least OpenMC version 0.12.0. There are some differences in the implementation and structuring of the depletion data, and each representation is not exchangable with the other, e.g. using an `openmc.deplete.Chain` inside this package in place of a `hydep.DepletionChain` is not supported

In [22]:
chain = hydep.DepletionChain.fromXml("../../chains/chain_endfb71.xml")

The `Manager` class is responsible for dictating the time schedule, e.g. time step size, constant or variable power, and substepping divisions. The latter is used to determine how many reduced-order transport solutions are performed at each time step. We will use a power of 174 W corresponding to a pin with height of 1 cm operating at 174 W/cm linear power.

Time steps are provided in units of days, and users can provide a single power to be used across all time steps, or one power per time step. We will instruct the framework to divide the first interval of five days into five, single day substeps. The reduced-order code will provide the flux solutions at days 2, 3, and 4 with this schedule. These substep fluxes will be used to deplete across a smaller time step, hopefully reducing computational time without penalizing accuracy by a substantial margin. The overall accuracy of your solution will be dependent on many factors, coarse step (distance between high-fidelity solutions) and substep size, operating power, and compositions in your problem.

In [23]:
manager = hydep.Manager(
    chain,
    daysteps=[5, 5],  # days
    power=174,  # W
    substepDivision=[5, 1],
)

Now the depletion side of the problem is handled, as the manager will be responsible for accepting reaction rates across all burnable materials, and passing back new compositions to the framework. By default, the 16-th order incomplete partial factorization form of CRAM [Pusa 2016](https://doi.org/10.13182/NSE15-26) will be used. The solver can be configured through `Manager.setDepletionSolver`

<div class="alert alert-info">
            
**See also**

* [Depletion reference](../api/depletion.rst)
* OpenMC depletion chain sources: https://openmc.org/depletion-chains/
            
</div>

## Mixed fidelity transport solutions

`hydep` was developed in order to facilitate efficient transport-depletion solutions by coupling high-fidelity and reduced-order transport solutions together. If one was to model depletion with a fine time scale, the computational resources required for an accurate and stable Monte Carlo solution can become burdensome. The simulation time can be reduced by exchanging some or many of the high-fidelity solutions with faster reduced-order simulations. The cost of this exchange is related to the accuracy of the reduced-order code.

One of the goals of `hydep` is to provide a modeling framework that can be understood by Monte Carlo and deterministic transport codes. This drives the geometry restrictions above, but it also - _in theory_ - allows minimal updates to switch between two high-fidelity codes and/or two reduced-order codes. This is enabled by the abstraction of the solvers away from the classes responsible for depleting and marching forward in time, similar to the design of `openmc.deplete`. We have our geometry defined, and the class responsible for updating compositions, now we must define the high-fidelity and reduced-order transport solvers.

`hydep` provides three submodules for interfacing with existing transport codes:

* `hydep.serpent` - [Interface for the Serpent Monte Carlo code](../api/serpent.rst)
* `hydep.simplerom` - [Solver that passes back most recent high-fidelity flux solution](../api/simplero.rst)
* `hydep.sfv` - [Interface for using the spatial flux variation method (SFV) as a reduced-order solver](../api/sfv.rst)

Here, we will use the standard Serpent solver (2.1.30+) in conjunction with the `SimpleROSolver` 

<div class="alert alert-warning">

**Warning**

The `hydep.serpent` module requires the `serpentTools` python package - https://serpent-tools.readthedocs.io/en/latest/
    
</div>

In [24]:
import hydep.serpent
import hydep.simplerom

In [25]:
highFi = hydep.serpent.SerpentSolver()

In [26]:
reduced = hydep.simplerom.SimpleROSolver()

Both classes do not require input arguments, but are configured through a custom settings interface

## Settings

`hydep` can be configured through an INI / CFG file via the `Settings` class. Attributes on the `Settings` class are passed directly to the various solvers and define both global parameters (boundary conditions, use of temporary directory) and solver-specific options (initial guess of $k$, data libraries, and particle statistics for Serpent).

In [27]:
settings = hydep.Settings()

In [28]:
settings.updateAll({
    "hydep": {
        "boundary conditions": "reflective",
        "use temp dir": True,
    },
    "hydep.serpent": {
        "active" : 10,
        "inactive": 5,
        "particles": 1e2,
        "generations per batch": 2,
        "executable": "sss2",
        "acelib": "sss_endfb7u.xsdata",
        "declib": "sss_endfb7.dec",
        "nfylib": "sss_endfb7.nfy",
    }
})

The goal of this approach is to reduce the amount of information that must be updated in order to switch between supported transport solvers. In theory, one could swap out a different high-fidelity solver, update a few lines of the configuration file, and the framework would be able to run without a problem. This is a difficult claim to prove, given only a single high-fidelity solver is supported at the moment. But the design of the interfaces and reliance on abstraction are indications that this claim has merit.

<div class="alert alert-info">
    
**See also**

* [Example configuration](../configuration.rst#example-configuration)
* [Settings interface](../api/settings.rst)
    
</div>

## Putting it all together

`hydep` provides [integrator classes](../api/integrators.rst) much like those provided in `openmc.deplete`, though not as extensive nor complicated. This is partly because the use of reduced-order transport solutions should allow smaller depletion steps without substantial increase in computational resources. Some higher-order schemes are provided, and other can be added in ways that mimic the `openmc.deplete` classes.

We will start with the basic `PredictorIntegrator`, using a single high-fidelity solution per coarse and substep. When constructing the integrators, we must provide all the simulation data: geometry, depletion schedule and chain, and transport solvers.

In [29]:
pred = hydep.PredictorIntegrator(model, highFi, reduced, manager)

We also apply our settings to the integrator, which will be passed to the main solution sequence.

In [30]:
pred.settings = settings

`hydep` allows users and developers to define their own data storage routine. By default, the `h5py` package [h5py]_ will be used to create a single output file containing simulation data. This is configured automatically if such storage is not defined prior to calling `Integrator.integrate`, and can also be configured with the `store` attribute

In [31]:
from hydep.hdfstore import HdfStore

In [32]:
pred.store = HdfStore("pincell-results.h5")  # file name is optional

We can finally run the simulation.

In [33]:
pred.integrate()

/home/drew/research/hydep/src/hydep/serpent/processor.py:769: DataWarning: The following isotopes did not have fission yields at an energy of 0.0253 eV, but were replaced by the closet set of provided yields: {902320: 500000.0, 922340: 500000.0, 922360: 500000.0, 922380: 500000.0}
  warnings.warn(


FailedSolverError:  -- Points added in neutron grid: 84
 95344.09c -- Points added in neutron grid: 57
 96242.09c -- Points added in neutron grid: 1396
 96243.09c -- Points added in neutron grid: 1601
 96244.09c -- Points added in neutron grid: 5565

Generating unionized energy grid:

 - Unionization performed without grid thinning
   between 1.00E-11 and 20.0 MeV.

 - Final neutron grid size: 1094282 points.

 - 25.05 Mb of memory allocated for grid data

OK.

Processing cross sections and ENDF reaction laws...



## References

[h5py] http://docs.h5py.org/en/stable/

[serpentTools] http://serpent-tools.readthedocs.io/en/latest/

[OpenMC] https://docs.openmc.org/en/latest/

[openmc.deplete] https://docs.openmc.org/en/latest/pythonapi/deplete.html#module-openmc.deplete